In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as pltb
from sklearn.model_selection import train_test_split


In [2]:
df=pd.read_csv('../Data/sample_labels.csv')

In [3]:
df=df[['Image Index','Finding Labels']]

In [4]:
df.head()

,Image Index,Finding Labels
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...
1,00000013_026.png,Cardiomegaly|Emphysema
2,00000017_001.png,No Finding
3,00000030_001.png,Atelectasis
4,00000032_001.png,Cardiomegaly|Edema|Effusion


In [5]:
df['Diseases'] = df['Finding Labels'].apply(lambda x: x.split('|'))

In [6]:
df.head()

,Image Index,Finding Labels,Diseases
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,"[Emphysema, Infiltration, Pleural_Thickening, ..."
1,00000013_026.png,Cardiomegaly|Emphysema,"[Cardiomegaly, Emphysema]"
2,00000017_001.png,No Finding,[No Finding]
3,00000030_001.png,Atelectasis,[Atelectasis]
4,00000032_001.png,Cardiomegaly|Edema|Effusion,"[Cardiomegaly, Edema, Effusion]"


In [7]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Labels'] = label_encoder.fit_transform(df['Finding Labels'])

In [8]:
df.head(10)

,Image Index,Finding Labels,Diseases,Labels
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,"[Emphysema, Infiltration, Pleural_Thickening, ...",190
1,00000013_026.png,Cardiomegaly|Emphysema,"[Cardiomegaly, Emphysema]",101
2,00000017_001.png,No Finding,[No Finding],234
3,00000030_001.png,Atelectasis,[Atelectasis],0
4,00000032_001.png,Cardiomegaly|Edema|Effusion,"[Cardiomegaly, Edema, Effusion]",88
5,00000040_003.png,Consolidation|Mass,"[Consolidation, Mass]",133
6,00000042_002.png,No Finding,[No Finding],234
7,00000057_001.png,No Finding,[No Finding],234
8,00000061_002.png,Effusion,[Effusion],154
9,00000061_019.png,No Finding,[No Finding],234


In [9]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
def construct_image_paths(image_folder, image_indices):
    return [os.path.join(image_folder, img) for img in image_indices]

image_folder = '../Data/images2'

train_image_paths = construct_image_paths(image_folder, train_df['Image Index'])
test_image_paths = construct_image_paths(image_folder, test_df['Image Index'])


In [11]:
import cv2
def load_and_preprocess_images(image_paths, labels):
    images = []
    for path, label in zip(image_paths, labels):
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
        img = cv2.resize(img, (224, 224))  
        img = img / 255.0  
        images.append(img)
    return np.array(images), labels


In [12]:
import tensorflow as tf
with tf.device('/device:GPU:0'):
  X_train, y_train = load_and_preprocess_images(train_image_paths, train_df['Labels'])
  X_test, y_test = load_and_preprocess_images(test_image_paths, test_df['Labels'])

2023-12-06 23:14:24.033900: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-06 23:14:24.034014: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-06 23:14:24.077395: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-06 23:14:24.191930: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-06 23:14:25.036773: W tensorflow/compiler/tf2

In [13]:
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [14]:
base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

2023-12-08 20:48:07.874603: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-08 20:48:07.912065: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-08 20:48:07.912207: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [15]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
for layer in base_model.layers:
    layer.trainable = False

In [17]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(label_encoder.classes_), activation='softmax')  
])

In [18]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [20]:
#history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test))

In [19]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [22]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13085288453462529641
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10465902592
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5022303359768225338
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2023-12-06 23:15:48.223974: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-06 23:15:48.224924: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-06 23:15:48.225462: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [23]:
import tensorflow as tf

print(f'\nTensorflow version = {tf.__version__}\n')
print(f'\n{tf.config.list_physical_devices("GPU")}\n')



Tensorflow version = 2.12.0


[]

